In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# 500K US Business Points
`civicknowledge.com-scorepoints-demosearch-1.1.1` Last Update: 2021-06-04T21:02:45

_Point locations for 500K US businesses, primarily shopping, entertainment, food and automomobile related._


Run the build.ipynb notebook to generate the points, if 
they don't already exist. THen build the packages with metapack.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [business_points](data/business_points.csv)**. US Business points

In [3]:
bpkg = mp.open_package('http://radius.civicknowledge.com.s3.amazonaws.com/businesslistdatabase.com-us_business.csv')
bpkg

# US Businesses
`businesslistdatabase.com-us_business-1.1.1` Last Update: 2021-04-09T23:19:03

_US Business list, reduced to point position and category_



This package processes a list of US businesses obtained from a
[private source](https://www.businesslistdatabase.com/). The
source data is low quality, so there is a lot of processing
to be done. Most importantly, many records have a conflict between 
the SIC and NAICS codes; these records are removed. 

## Building

The package can take muh longer to build than the timeout for 
running a Jupyter notebook, so the Jupyter notebook caches
intermediate stages of the process. Run the upyter notebook directly
first, then build the package.
## Documentation Links

* [Documentation Page](http://s3.amazonaws.com/radius.civicknowledge.com/businesslistdatabase.com-us_business-1.1.1/index.html) 

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [businesses_cat](http://s3.amazonaws.com/radius.civicknowledge.com/businesslistdatabase.com-us_business-1.1.1/data/businesses_cat.csv)**. Businesses categorized to shop, auto, food, ent

## References
<ul><li> <strong>us_business</string>, <em>data/us_businesses.zip</em>. List of US Businesses</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><li> <strong>states</string>, <em>censusgeo://2020/5/US/state</em>. US States</li><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong><a href="https://www.census.gov/eos/www/naics/2012NAICS/2012_NAICS_Index_File.xls">naics_index_2012</a></strong>. NAICS index file, 2012</li><li> <strong><a href="https://www.census.gov/eos/www/naics/2012NAICS/2-digit_2012_Codes.xls">naics_index_2012_26</a></strong>. NAICS index file, 2012, 2 to 6 digit codes</li><li> <strong><a href="https://www.census.gov/naics/concordances/1997_NAICS_to_1987_SIC.xls">naics_sic</a></strong>. NAICS to SIC crosswalk, maybe?</li><ul>

In [4]:
cbsa = bpkg.reference('cbsa').geoframe()
bc= bpkg.resource('businesses_cat').geoframe()
t = gpd.sjoin(bc, cbsa.to_crs(4326))
df = t[~t.geoid.isnull()]

In [5]:
# Only need a subset of columns. 
t = df.reset_index()[['index', 'geoid', 'geometry']].copy().rename(columns={'index':'idx'})

# Use geohashes to ensure only one point within each approx 150mx150m region
import libgeohash as gh 
t['geohash'] = t.geometry.apply(lambda v: gh.encode(v.y, v.x, precision = 7) )

print(len(t))
t = t.drop_duplicates(subset=['geohash'])
print(len(t))

# Sample 100 records from every CBSA to ensure that in the final set, 
# each CBSA has at least 100. We'll remove duplicates later. 
def sample(g):
    try:
        return g.sample(100)
    except ValueError:
        return None

df100 = t.groupby('geoid').apply(sample).reset_index(drop=True)

# Sample the ballance of the 500k
df400 = t[~t.idx.isin(df100.idx)].sample(500_000-len(df100))

# combine the sets together. 
df500 = pd.concat([df400, df100]).sort_values('geoid').drop_duplicates(subset=['geohash'])


len(df500)

2126724
1009478


500000

In [6]:
len(df500)

500000

In [7]:
#df500.to_csv('../data/business_points.csv')